In [ ]:
!pip install ecpy
!pip install pycryptodome

  Using cached https://files.pythonhosted.org/packages/e8/35/4a113189f7138035a21bd255d30dc7bffc77c942c93b7948d2eac2e22429/ECPy-1.2.5-py3-none-any.whl


In [ ]:
import math
import timeit
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json
API_URL = 'http://cryptlygos.pythonanywhere.com'

stuID = 25389

In [ ]:
#HERE CREATE A LONG TERM KEY
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b


In [ ]:



#key generation
sA = randint(1,n-1) #long term private key
lprivate = sA
QA = (lprivate) * P
lkey = QA #long term public key

#signature generation
m = str(stuID).encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))

h_=h.digest()
h=int.from_bytes(h_, byteorder='big')
h = h % n

s = ((lprivate * h) + k)  % n
signature = (h,s)

#signature verification
V = (s*P) - (h*QA)
v = (V.x) % n
v = v.to_bytes((v.bit_length()+7)//8, byteorder='big')
Z = m+v 

l = (SHA3_256.new(Z))
l_=l.digest()
h2=int.from_bytes(l_, byteorder='big')
h2 = h2 % n

if h2 == h:
  print("Signature verified!")
else:
  print("Signature denied!")

#server's long term key
QSer_long = Point(0xc1bc6c9063b6985fe4b93be9b8f9d9149c353ae83c34a434ac91c85f61ddd1e9 , 0x931bd623cf52ee6009ed3f50f6b4f92c564431306d284be7e97af8e443e69a8c, E)
print("long term private: ", lprivate, " long term public", lkey)


Signature verified!
ID: 25389 h: 38259768464250379635040497775890139855882915235214709993057000913918828773916 s: 87850353099225359665562494505080637981331677900847642053424518055570024211745 LKEY.X: 13619625218732843456028323071152542955109936481024470844470356748070850858941 LKEY.Y: 42064519950064156800251459524511966204349551417264822203728114954854536303590
long term private:  45917922352925957393588137970243412602337613049636322288077465947682944756984  long term public (0x1e1c6f48505619df3ba0367b275130e0e7bf0ad9b2637067ea2707a7d4770fbd , 0x5cffacb13eb2e3bde7247fd8d245725c9d2c05d3c527a7303f25ff5ac917d3e6)


In [ ]:
long_term_private = 45917922352925957393588137970243412602337613049636322288077465947682944756984      
long_term_public = Point(0x1e1c6f48505619df3ba0367b275130e0e7bf0ad9b2637067ea2707a7d4770fbd , 0x5cffacb13eb2e3bde7247fd8d245725c9d2c05d3c527a7303f25ff5ac917d3e6 , E)
h = 38259768464250379635040497775890139855882915235214709993057000913918828773916 
s = 87850353099225359665562494505080637981331677900847642053424518055570024211745 
# HERE GENERATE A EPHEMERAL KEY 
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

ephemeral_sa = randint(1,n-1)
ephemeral_QA = (ephemeral_sa) * P
ekey = ephemeral_QA 

In [ ]:
try:
  #REGISTRATION
  mes = {'ID':stuID, 'h': h, 's': s, 'LKEY.X': lkey.x, 'LKEY.Y': lkey.y}
  response = requests.put('{}/{}'.format(API_URL, "RegStep1"), json = mes)		
  if((response.ok) == False): raise Exception(response.json())
  print(response.json())

  print("Enter verification code which is sent to you: ")	
  code = int(input())

  mes = {'ID':stuID, 'CODE': code}
  response = requests.put('{}/{}'.format(API_URL, "RegStep3"), json = mes)
  if((response.ok) == False): raise Exception(response.json())
  print(response.json())


  #STS PROTOCOL

  mes = {'ID': stuID, 'EKEY.X': ekey.x, 'EKEY.Y': ekey.y}
  response = requests.put('{}/{}'.format(API_URL, "STSStep1&2"), json = mes)
  if((response.ok) == False): raise Exception(responce.json())
  res=response.json()

  # calculate T,K,U
  skeyx = res["SKEY.X"]
  skeyy = res["SKEY.Y"]
  QB = Point(skeyx, skeyy, E)
  T = ephemeral_sa * QB
  t_x = str(T.x)
  t_y = str(T.y)
  string = "BeYourselfNoMatterWhatTheySay"
  U = t_x + t_y + string
  u_ = U.encode()
  K = SHA3_256.new(u_)
  K = K.digest()

  # Sign Message
  W1 = str(ekey.x) + str(ekey.y) + str(QB.x) + str(QB.y) 
  k = randint(1,n-1)
  R2 = k * P
  r2 = (R2.x) % n
  r2 = r2.to_bytes((r2.bit_length() + 7) // 8, 'big')
  T2 = W1.encode() + r2
  h2 = (SHA3_256.new(T2))
  h2_ = h2.digest()
  h2 = int.from_bytes(h2_, byteorder='big')
  h2 = h2 % n
  sl = long_term_private

  s2 = ((sl * h2) + k) % n
  (Siga_s, Siga_h) = (s2, h2)
  
  # Encryption
  my_message = "s" + str(Siga_s) + "h" + str(Siga_h)
  plaintext = my_message.encode()
  cipher = AES.new(K, AES.MODE_CTR)
  Y1 = cipher.encrypt(plaintext)

  ctext = cipher.nonce[0:8] + Y1
  ctext = int.from_bytes(ctext, byteorder='big')
  ###Send encrypted-signed keys and retrive server's signed keys
  mes = {'ID': stuID, 'FINAL MESSAGE': ctext}
  response = requests.put('{}/{}'.format(API_URL, "STSStep4&5"), json=mes)
  if ((response.ok) == False): raise Exception(response.json())
  ctext = response.json()

  #Decrypt 
  ctext = ctext.to_bytes((ctext.bit_length()+7)//8, byteorder='big')
  cipher = AES.new(K, AES.MODE_CTR, nonce=ctext[0:8])
  Y2 = cipher.decrypt(ctext[8:])
  temp1 = str(Y2).find("s")
  temp2 = str(Y2).find("h")
  sigb_s = str(Y2)[temp1+1:temp2]
  sigb_h = str(Y2)[temp2+1:]
  sigb_h = sigb_h[:-1]
  sigb_s = int(sigb_s)
  sigb_h = int(sigb_h)

  #verify
  W2 = str(QB.x) + str(QB.y) + str(ekey.x) + str(ekey.y) 
  m = W2.encode()
  V = (sigb_s * P) - (sigb_h*QSer_long)  
  v = (V.x) % n 
  v = v.to_bytes((v.bit_length()+7)//8, byteorder='big')
  Z = m+v

  l = (SHA3_256.new(Z))
  l_=l.digest()
  h2=int.from_bytes(l_, byteorder='big')
  h2 = h2 % n

  if h2 == sigb_h:
    print("Signature verified!")
  else:
    print("Signature denied!")

  #get a message from servesr for 
  mes = {'ID': stuID}
  response = requests.get('{}/{}'.format(API_URL, "STSStep6"), json=mes)
  server_cipher = response.json()         

  #Decrypt
  server_cipher = server_cipher.to_bytes((server_cipher.bit_length()+7)//8, byteorder='big')
  cipher = AES.new(K, AES.MODE_CTR, nonce=server_cipher[0:8])
  message = cipher.decrypt(server_cipher[8:])
  message = message.decode()
  print("Message from server: " , message)
  RAND = message[-8:]
  message = message[0:len(message)-8]

  #Add 1 to random to create the new message and encrypt it
  RAND = int(RAND) + 1
  plaintext =  message + str(RAND)
  print("MessageToServer: " ,plaintext)
  plaintext = plaintext.encode()
  cipher = AES.new(K, AES.MODE_CTR)
  ct = cipher.nonce[0:8] + cipher.encrypt(plaintext)
  ct = int.from_bytes(ct, byteorder='big')
  W4 = RAND
  #send the message and get response of the server
  mes = {'ID': stuID, 'ctext': ct}
  response = requests.put('{}/{}'.format(API_URL, "STSStep7&8"), json = mes)
  ctext= response.json()         

  server_cipher = ctext.to_bytes((ctext.bit_length()+7)//8, byteorder='big')
  cipher = AES.new(K, AES.MODE_CTR, nonce=server_cipher[0:8])
  message = cipher.decrypt(server_cipher[8:])
  message = message.decode()
  print("Expected message from server: ", message)

except Exception as e:
  print(e)


Signature verified!
Message from server:  When you read this message I'll be far away. 16731112
MessageToServer:  When you read this message I'll be far away. 16731113
Expected message from server:  When you read this message I'll be far away. 16731114


In [ ]:
#BURDAN ITIBAREN PHASE 2

In [ ]:
import math
import timeit
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json
API_URL = 'http://cryptlygos.pythonanywhere.com'

stuID =  25389

E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b
#create a long term key
long_term_private = 45917922352925957393588137970243412602337613049636322288077465947682944756984      
QCli_long = Point(0x1e1c6f48505619df3ba0367b275130e0e7bf0ad9b2637067ea2707a7d4770fbd , 0x5cffacb13eb2e3bde7247fd8d245725c9d2c05d3c527a7303f25ff5ac917d3e6 , E)
m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h=int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n

#server's long term key
QSer_long = Point(0xc1bc6c9063b6985fe4b93be9b8f9d9149c353ae83c34a434ac91c85f61ddd1e9 , 0x931bd623cf52ee6009ed3f50f6b4f92c564431306d284be7e97af8e443e69a8c, E)



In [ ]:

####Register Long Term Key
"""mes = {'ID':25389, 'H': h, 'S': s, 'LKEY.X': QCli_long.x, 'LKEY.Y': QCli_long.y}
response = requests.put('{}/{}'.format(API_URL, "RegLongRqst"), json = mes)
print(response.json())
code = input()

mes = {'ID':25389, 'CODE': code}
response = requests.put('{}/{}'.format(API_URL, "RegLong"), json = mes)
print(response.json())"""


def key_generator():
  E = Curve.get_curve('secp256k1')
  n = E.order
  p = E.field
  P = E.generator
  a = E.a
  b = E.b
  sA = randint(1,n-1)
  QA = (sA) * P
  return sA, QA

sA0, QA0 = key_generator()
sA1, QA1 = key_generator()
sA2, QA2 = key_generator()
sA3, QA3 = key_generator()
sA4, QA4 = key_generator()
sA5, QA5 = key_generator()
sA6, QA6 = key_generator()
sA7, QA7 = key_generator()
sA8, QA8 = key_generator()
sA9, QA9 = key_generator()

#signature generation
def sign_message(QA, long_priv):
  E = Curve.get_curve('secp256k1')
  n = E.order
  p = E.field
  P = E.generator
  a = E.a
  b = E.b

  m  = str(QA.x) + str(QA.y)
  m = m.encode()
  k = randint(1,n-1)
  R = k*P
  r = (R.x) % n
  r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
  T = m+r 
  h = (SHA3_256.new(T))
  h_= h.digest()
  h=int.from_bytes(h_, byteorder='big')
  h = h % n

  s = ((long_priv * h) + k)  % n
  return s, h

s0, h0 = sign_message(QA0, long_term_private)
s1, h1 = sign_message(QA1, long_term_private)
s2, h2 = sign_message(QA2, long_term_private)
s3, h3 = sign_message(QA3, long_term_private)
s4, h4 = sign_message(QA4, long_term_private)
s5, h5 = sign_message(QA5, long_term_private)
s6, h6 = sign_message(QA6, long_term_private)
s7, h7 = sign_message(QA7, long_term_private)
s8, h8 = sign_message(QA8, long_term_private)
s9, h9 = sign_message(QA9, long_term_private)

In [ ]:
#BURAYI YUKLEMEK GEREK MI???
###delete ephemeral keys
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n


mes = {'ID': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "RstEKey"), json = mes)
print(response.json())

200


In [ ]:
print("sA0: ", sA0, "QA0: ", QA0, "s0: ", s0, "h0: ", h0)
print("sA1: ", sA1, "QA1: ", QA1, "s1: ", s1, "h1: ", h1)
print("sA2: ", sA2, "QA2: ", QA2, "s2: ", s2, "h2: ", h2)
print("sA3: ", sA3, "QA3: ", QA3, "s3: ", s3, "h3: ", h3)
print("sA4: ", sA4, "QA4: ", QA4, "s4: ", s4, "h4: ", h4)
print("sA5: ", sA5, "QA5: ", QA5, "s5: ", s5, "h5: ", h5)
print("sA6: ", sA6, "QA6: ", QA6, "s6: ", s6, "h6: ", h6)
print("sA7: ", sA7, "QA7: ", QA7, "s7: ", s7, "h7: ", h7)
print("sA8: ", sA8, "QA8: ", QA8, "s8: ", s8, "h8: ", h8)
print("sA9: ", sA9, "QA9: ", QA9, "s9: ", s9, "h9: ", h9)

sA0:  82543384283391623622849791820575630198488103744083980548928116053600005759945 QA0:  (0xfb2b48d228ddbcdc8c3a6b957de22757f1027af5a4dd85c7aff17a7deb1b4958 , 0x667a0d8c974d9a81c5d5e598613839eac327782cd3588c8d423d136352468c7) s0:  82720513417142409764978006740305158672497742081804379812192344933328242570816 h0:  28288362648394007017447150617984660347988209942587627533123706027538343303625
sA1:  115606596987806643177008702969844986728848373589906641795160548218968247600803 QA1:  (0x9c75d3d58e0d870ee72d7b0081b82f17956cc718b81c84fd26b525baa79f61df , 0x95aee8441c1afc486a444cc3b11cdcd1c66e4d7291e2c8941d5694bd2993eef8) s1:  69839503343330213539255704641105853118984003631558227104930340515924177221774 h1:  101609936301700590093394863128365629640961790710818979346265718663740752324068
sA2:  102051085025409459694123872404617753804917400802047317073885463316937097769392 QA2:  (0xd860e35190dbbc6f7324e99888820c7860b469c7ae334966881c464057e8e6d2 , 0xd556004b37d8fde5a275f67e6e949a512a1ba3ae8060f6df

In [ ]:
#send ephemeral key
mes = {'ID': stuID, 'KEYID': 0 , 'QAI.X': QA0.x, 'QAI.Y': QA0.y, 'Si': s0, 'Hi': h0}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 1 , 'QAI.X': QA1.x, 'QAI.Y': QA1.y, 'Si': s1, 'Hi': h1}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 2 , 'QAI.X': QA2.x, 'QAI.Y': QA2.y, 'Si': s2, 'Hi': h2}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 3 , 'QAI.X': QA3.x, 'QAI.Y': QA3.y, 'Si': s3, 'Hi': h3}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 4 , 'QAI.X': QA4.x, 'QAI.Y': QA4.y, 'Si': s4, 'Hi': h4}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 5 , 'QAI.X': QA5.x, 'QAI.Y': QA5.y, 'Si': s5, 'Hi': h5}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 6 , 'QAI.X': QA6.x, 'QAI.Y': QA6.y, 'Si': s6, 'Hi': h6}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 7 , 'QAI.X': QA7.x, 'QAI.Y': QA7.y, 'Si': s7, 'Hi': h7}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 8 , 'QAI.X': QA8.x, 'QAI.Y': QA8.y, 'Si': s8, 'Hi': h8}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 9 , 'QAI.X': QA9.x, 'QAI.Y': QA9.y, 'Si': s9, 'Hi': h9}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())



Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key
Key index is not correct. You need to send 9th key


In [ ]:
#GET 5 MESSAGES BELOW

In [ ]:
#Receiving Messages
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n

s = ((long_term_private * h) + k)  % n
mes = {'ID_A': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)
print(response.json())

{'IDB': 18007, 'KEYID': '6', 'MSG': 38685261194367170781688761115296361990639311618804121882833982419750728007153383696441454815007932406334387156570572676941647420018101183428781171464955970252097772549743871815282892422001516633121550, 'QBJ.X': 10210105890238661899064609876853679076402526757997419024605930603431240443, 'QBJ.Y': 107708383502831461764679818825890967236590187762417571595086386980206853942988}


In [ ]:
#Receiving Messages
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n

s = ((long_term_private * h) + k)  % n
mes = {'ID_A': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)
print(response.json())

{'IDB': 18007, 'KEYID': '1', 'MSG': 40430275554905575658647265090516349925438261918962748994669174535274250817601268234441731203463177318852875000590496549412481161478338085322617181044382451151914355989374576914368971596005856815908077, 'QBJ.X': 32636733385320361812730116183108216112668701450022373767616965055565622363342, 'QBJ.Y': 98805030337088261303687121196290041334423983009141542664088673140955547989082}


In [ ]:
#Receiving Messages
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n

s = ((long_term_private * h) + k)  % n
mes = {'ID_A': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)
print(response.json())

{'IDB': 18007, 'KEYID': '2', 'MSG': 46957132003909831621362563400219459478279428819147275968748836541036758098502340728879311464087158785988159511390832471253939556787478661263268233844485104489616512459805070793519494174779768010521017, 'QBJ.X': 48421277388483113352796411757482422327009369565755376719093590240488632794974, 'QBJ.Y': 45793766870943501013787128727136037790926467608586803575527565543314585414372}


In [ ]:
#Receiving Messages
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n

s = ((long_term_private * h) + k)  % n
mes = {'ID_A': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)
print(response.json())


{'IDB': 18007, 'KEYID': '3', 'MSG': 8613400940584567846211301790391552653323274764595594923107761604050568775380330040833421170917090159578623907282823200856134045715056840292780272297197522299054250865284521679605327324438806883757895, 'QBJ.X': 110941973471089384064366641208277936190837296188861716320585526375853061531982, 'QBJ.Y': 89033303002974604176376089065035288149962165466988384265564406712841984994869}


In [ ]:
#Receiving Messages
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n

s = ((long_term_private * h) + k)  % n
mes = {'ID_A': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)
print(response.json())

{'IDB': 18007, 'KEYID': '4', 'MSG': 25441463477105774006518871665545698415413456690181036911890493897527670375953998395217970206615576473303607676562415974441731269752939266785391848723261812521636623137483947550020581200348715727598672, 'QBJ.X': 20444406111849703453681648722845849248601932765397280751804819667293278140642, 'QBJ.Y': 29551710782173199133733203477680974121259083892407943964389582007362559282929}


In [ ]:
sA0, QA0, s0, h0 = 82543384283391623622849791820575630198488103744083980548928116053600005759945 , (0xfb2b48d228ddbcdc8c3a6b957de22757f1027af5a4dd85c7aff17a7deb1b4958 , 0x667a0d8c974d9a81c5d5e598613839eac327782cd3588c8d423d136352468c7) ,  82720513417142409764978006740305158672497742081804379812192344933328242570816 ,  28288362648394007017447150617984660347988209942587627533123706027538343303625
sA1, QA1, s1, h1 = 115606596987806643177008702969844986728848373589906641795160548218968247600803 , (0x9c75d3d58e0d870ee72d7b0081b82f17956cc718b81c84fd26b525baa79f61df , 0x95aee8441c1afc486a444cc3b11cdcd1c66e4d7291e2c8941d5694bd2993eef8) , 69839503343330213539255704641105853118984003631558227104930340515924177221774 ,  101609936301700590093394863128365629640961790710818979346265718663740752324068
sA2, QA2, s2, h2 = 102051085025409459694123872404617753804917400802047317073885463316937097769392 ,  (0xd860e35190dbbc6f7324e99888820c7860b469c7ae334966881c464057e8e6d2 , 0xd556004b37d8fde5a275f67e6e949a512a1ba3ae8060f6dfa14326d3d75b09f0) ,  25313671591917344271177305335336361157015944766668569089714427207916581707292 , 85041708475274993481341250802219137891842380961309803233319787599409017391639
sA3, QA3, s3, h3 = 100056109020662515818180407479702142632948903298738057252512779433430994800057 , (0xe3cf5c7ad5ad2a36c84938906d4aa708227c5a97fbfe13018a0990a9c134e935 , 0xc1f9670d9b749c75b35d236019077e4f143f72f8809a2a6dec1f15ff71eaec71) ,  12220845454396965748757270544193118121522077850760890261175597238247962718875 ,  94578874030025145254647363571532083752731485510654601665468074913304690323071
sA4, QA4, s4, h4 = 18716412006718086656232009765038629499379530035782610360024808011470537630902 ,  (0x987448e6eb425903f8d5835ad9ff5187cfdd867e6035cea4048ecf4e0c604c17 , 0xdf813d1568549e03bc7e0f84f83301e1db6efa08666807c6cad75cdd6fbadfe7) , 67719794330328712697472735519011489598231768956822624327105235541024499898464 , 84366196511542838752928152180347402423769995800123882128789580267246784963237
sA5, QA5, s5, h5 = 15042697023948426675811855476733496962443938328832977897233540956754308576174 , (0x1f57be86cb78b10d68b1957b51036a42eb601d3bb6ae94cdcb76959f1bc2a88e , 0x9b2f838fee9470eabc0b7e8b2b04f4f61928afd39954a4688346807c0fe92d55) ,  110079807378333116436796367823436796926261988285828966411596706472573964588969 , 41609810393756258132183153247179668100474364794754027679933459992404362636285
sA6, QA6, s6, h6 = 85386185288710580090357355757582342229569303938438416817559036827266026802050 , (0xddebe7c0f76edfce6bb63d99fc590134c749ba899d9b4767c8cdaee1bfb2cc96 , 0xca254e472faf2399e80c106f01ce085e4c431abcc5296a54f73aa419e48b0494) ,  115301252969220162702759118770155118400502262604399135395804531859184143358644 ,  75806659055906352911251482067816076215754191423650838948003294913519708850391
sA7, QA7, s7, h7 = 74318440188815669783690315293288987409084669343154606118973843935372322413271 ,  (0x9800b079e24f0a574d38b8d195624c9426ca9fe20508e1458bfd07b58e5cfd98 , 0xe1dc42bec2d55922ccc4e6e3705413697f6eb8ea3028b092462e42df63cbeaeb) , 30152059764832952400321754789840693401302065942683450851027577258385208914251 ,  111394653762927129010303073119565970311964087596160152347469730702891499399844
sA8, QA8, s8, h8 = 42425719425855273331959379146980108213255799433913507275942363572766467048509 , (0x49eaa223c864149967336dbeb14c40a90523f7bb470632498c06d4b831c61475 , 0x72ae161281f1962c5db0897c26f1d73212748d3a3c33db161b37bdee6c2e2c10) ,  20182049496958868223225827922582702725632795710943759746446061012468988393933 ,  67103133808441375344629366569779447084546147067656205685755619731900349035299
sA9, QA9, s9, h9 = 13704814155367561487837776297157103536421134861131298841855928923412262614639 ,  (0x87e09c9fdb208578a24c29dc9aa9d68219dbfb3fb528cab1f99e950ab92e3ee9 , 0x53d6531bb6942abc39925479f8df42c33da546190a20e0206a6f49a63b4d62ba) , 27457068836150299574017050299947955038373640682067760462728793427362932325773 , 68779957237338393836158755926511756344537206498567792148828479750595895575298

In [ ]:
#decrypt messages
from Crypto.Hash import HMAC, SHA256
#session keys
def session_keys_and_decrypt(message, si, Qj):
  T = si * Qj
  U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
  U = U.encode()
  KENC_AB = (SHA3_256.new(U))
  KENC_AB_ = KENC_AB.digest()
  KMAC_AB = (SHA3_256.new(KENC_AB_))
  KMAC_AB_ = KMAC_AB.digest()
  ctext = message.to_bytes((message.bit_length()+7)//8, byteorder='big')
  mac = ctext[-32:]
  ctext = ctext[0:-32]
  cipher = AES.new(KENC_AB_, AES.MODE_CTR, nonce=ctext[0:8])
  dtext = cipher.decrypt(ctext[8:])
  h = HMAC.new(KMAC_AB_ , digestmod=SHA256)
  h.update(ctext[8:])
  if mac == h.digest():
    print("The message ", message.to_bytes((message.bit_length()+7)//8, byteorder='big'), "is authentic")
    return dtext
  else:
    return "Not authentic"

The message  b'f\xe0\x0fO\n\x16\x8a\xee\xdbn+\x13n\xd0\xbd?\xc2\x19.<\xc7\x8dU\xc5\xa5i93\xc1\xecT\xf8\xd2[Bq\x12\x8d\xeey\x89\xf8\xc4\xf6\xbd\xe6\xf4#\xee\x8dn&\x93\xb7\x8b\x9a\xbb<\x1e\x06\xce\xb9m\x9b\xb0\xd9\xb0\x0c\xe8\xc4eb\xaf\xa1f\xb9\xab\xb0\xd4\x94)C\xa4' is authentic


'#send decrypted messages to server\nmes = {\'ID_A\': stuID, \'DECMSG\': h}\nresponse = requests.put(\'{}/{}\'.format(API_URL, "Checker"), json = mes)'

In [ ]:
#for KEYID:0 
msg0 = 17720794446848774991130155448282084288791104527069307820855503813929729016895914453518623392750481403203719599348495602147840350218544338723731834039818453146757642666430564098313414639364042369377010
QB0 = Point(94532122962867511185800589273587440472361078076014591619662504450715136624028, 23378557078196776373888889771714683320279156677649776442452558438633034068001,E)
message_0 = session_keys(msg0,sA0,QB0)
h = message_0.decode()
#send decrypted messages to server
mes = {'ID_A': stuID, 'DECMSG': h}
response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)
print(response.json())

The message  b';D\x1cKF\x98$\xd4o\xd8\x8a\xb6\xd6\x12\xd7\xe0!\x1b|*n\xc3\x9e\x8d\x0c\x08\x02\xec\x8d\xda\xae\xc7\xb9\xa47\x06\xbem\x10?km\x92C /S\x87W(\xefg5{5\x9d\x8cW\xab\xab\xbe\xfa\x11\xce\x10\x92\xac\xb0\xa4YU\x0b\xc8\xa4\xbe\xc10\xae\xed\xaa\xed\xa2\xf2' is authentic
https://www.youtube.com/watch?v=Q8Tiz6INF7I
Correct! Did you like the song??


In [ ]:
#for KEYID:1
msg1 = 40430275554905575658647265090516349925438261918962748994669174535274250817601268234441731203463177318852875000590496549412481161478338085322617181044382451151914355989374576914368971596005856815908077
QB1 = Point(32636733385320361812730116183108216112668701450022373767616965055565622363342, 98805030337088261303687121196290041334423983009141542664088673140955547989082, E)
message_1 = session_keys(msg1,sA1,QB1)
h = message_1.decode()
#send decrypted messages to server
mes = {'ID_A': stuID, 'DECMSG': h}
response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)
print(response.json())

The message  b"\x877i\x98,\x17\x9c\x8c\xf3\x9e\x86,d\xdd\x92f!`6\x96\x1e\xf2\xaaU~\x88\x8c\n\xbe\xfe\xd3D0<l(\x89\x0e\x02\xa5\xa6\xc1R\xba\xd4\x7f1Y\\a_\x84\xba\xd4\xf8[`\xff\xdb\xc2\x98X'\xb8\r\xef0S(\x81\xbd\xe2\xf1\x17r\x92\x1cw\x85\x83\xdc\x84\xed" is authentic
Correct! Did you like the song??


In [ ]:
#for KEYID:2
msg2 = 46957132003909831621362563400219459478279428819147275968748836541036758098502340728879311464087158785988159511390832471253939556787478661263268233844485104489616512459805070793519494174779768010521017
QB2 = Point(48421277388483113352796411757482422327009369565755376719093590240488632794974, 45793766870943501013787128727136037790926467608586803575527565543314585414372, E)
message_2 = session_keys(msg2,sA2,QB2)
h = message_2.decode()
#send decrypted messages to server
mes = {'ID_A': stuID, 'DECMSG': h}
response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)
print(response.json())

The message  b"\x9d\x0b\x8c\x17\xc6\xed&\xff\x197L\xda!s\xe7\xe5\xf2yC.&\x92\xa9l\xd3I'9\\\x17\xebP]k\x90}u\x0b0f\xf3\xe7t\xe9\xb9f\x16\xe0\xd6.\x0cHjn{\xc5*\xa2\xed\x98Or\x1e\x80M\x86\x13\r\xf5V\x9c/M\xcbwt\x87\xcda\xa4\x899\xb9" is authentic
Correct! Did you like the song??


In [ ]:
#for KEYID:3
msg3 = 8613400940584567846211301790391552653323274764595594923107761604050568775380330040833421170917090159578623907282823200856134045715056840292780272297197522299054250865284521679605327324438806883757895
QB3 = Point(110941973471089384064366641208277936190837296188861716320585526375853061531982, 89033303002974604176376089065035288149962165466988384265564406712841984994869, E)
message_3 = session_keys(msg3,sA3,QB3)
h = message_3.decode()
#send decrypted messages to server
mes = {'ID_A': stuID, 'DECMSG': h}
response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)
print(response.json())

The message  b"\x1c\xce\x95EUQw<'\x88\x88\xb8QF\xd6MK,\x88\x11`|46H\r\x19r)l\xb5uN\xfd\xdf\xf7l9\xb3\xbd9\x0b\xfcH 07p\x80\x9b\xcd\xbf\xf9Z\xd1b\x03\xc2\xb8U\xbe\x0f^\x9cr\xf5\xb9y\xb0\xf7\xfdc\xd0Y\xf6\x8a\x8a\xe2\xfd[\x84oG" is authentic
Correct! Did you like the song??


In [ ]:
#for KEYID:4
msg4 = 25441463477105774006518871665545698415413456690181036911890493897527670375953998395217970206615576473303607676562415974441731269752939266785391848723261812521636623137483947550020581200348715727598672
QB4 =  Point(20444406111849703453681648722845849248601932765397280751804819667293278140642, 29551710782173199133733203477680974121259083892407943964389582007362559282929, E)
message_4 = session_keys(msg4,sA4,QB4)
h = message_4.decode()
#send decrypted messages to server
mes = {'ID_A': stuID, 'DECMSG': h}
response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)
print(response.json())

The message  b"U\x16[\xc0\nT\x8a0\xd6\xae\x8f\n\x95\x10\xeb\xc2f\x03%\xf6\x87\xb1\x9b\x9f\xbf\x9b\xb5\xcc\xdf5\x0c\xcb\xfah\xb0'\x1e\x97R\x17F8\xbd\xfe\xfevi<\x03\xceZ\xcf\xe5Q\xec\xf1\xb6FS]yZh\xff\x0e$\x85\xbf(\xf0,\x99\xe9V\xb94\xc1\xc4_\x98\x9a4P" is authentic
Correct! Did you like the song??


In [ ]:
#PHASE 3
import math
import timeit
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, SHA256, HMAC
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import hashlib, hmac, binascii
import json
API_URL = 'http://cryptlygos.pythonanywhere.com'

stuID =  25389
stuID_B = 25334

E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

#create a long term key
long_term_private = 45917922352925957393588137970243412602337613049636322288077465947682944756984      
QCli_long = Point(0x1e1c6f48505619df3ba0367b275130e0e7bf0ad9b2637067ea2707a7d4770fbd , 0x5cffacb13eb2e3bde7247fd8d245725c9d2c05d3c527a7303f25ff5ac917d3e6 , E)
m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h=int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n

####Register Long Term Key

"""mes = {'ID':25389, 'H': h, 'S': s, 'LKEY.X': QCli_long.x, 'LKEY.Y': QCli_long.y}
response = requests.put('{}/{}'.format(API_URL, "RegLongRqst"), json = mes)
print(response.json())

code = int(input())

mes = {'ID':25389, 'CODE': code}
response = requests.put('{}/{}'.format(API_URL, "RegLong"), json = mes)
print(response.json())"""


'mes = {\'ID\':25389, \'H\': h, \'S\': s, \'LKEY.X\': QCli_long.x, \'LKEY.Y\': QCli_long.y}\nresponse = requests.put(\'{}/{}\'.format(API_URL, "RegLongRqst"), json = mes)\nprint(response.json())\n\ncode = int(input())\n\nmes = {\'ID\':25389, \'CODE\': code}\nresponse = requests.put(\'{}/{}\'.format(API_URL, "RegLong"), json = mes)\nprint(response.json())'

In [ ]:
#create new ephemeral keys
def key_generator():
  E = Curve.get_curve('secp256k1')
  n = E.order
  p = E.field
  P = E.generator
  a = E.a
  b = E.b
  sA = randint(1,n-1)
  QA = (sA) * P
  return sA, QA

sA0, QA0 = key_generator()
sA1, QA1 = key_generator()
sA2, QA2 = key_generator()
sA3, QA3 = key_generator()
sA4, QA4 = key_generator()
sA5, QA5 = key_generator()
sA6, QA6 = key_generator()
sA7, QA7 = key_generator()
sA8, QA8 = key_generator()
sA9, QA9 = key_generator()

#signature generation
def sign_message(QA, long_priv):
  E = Curve.get_curve('secp256k1')
  n = E.order
  p = E.field
  P = E.generator
  a = E.a
  b = E.b

  m  = str(QA.x) + str(QA.y)
  m = m.encode()
  k = randint(1,n-1)
  R = k*P
  r = (R.x) % n
  r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
  T = m+r 
  h = (SHA3_256.new(T))
  h_= h.digest()
  h=int.from_bytes(h_, byteorder='big')
  h = h % n

  s = ((long_priv * h) + k)  % n
  return s, h

s0, h0 = sign_message(QA0, long_term_private)
s1, h1 = sign_message(QA1, long_term_private)
s2, h2 = sign_message(QA2, long_term_private)
s3, h3 = sign_message(QA3, long_term_private)
s4, h4 = sign_message(QA4, long_term_private)
s5, h5 = sign_message(QA5, long_term_private)
s6, h6 = sign_message(QA6, long_term_private)
s7, h7 = sign_message(QA7, long_term_private)
s8, h8 = sign_message(QA8, long_term_private)
s9, h9 = sign_message(QA9, long_term_private)

In [ ]:
print("sA0: ", sA0, "QA0: ", QA0, "s0: ", s0, "h0: ", h0)
print("sA1: ", sA1, "QA1: ", QA1, "s1: ", s1, "h1: ", h1)
print("sA2: ", sA2, "QA2: ", QA2, "s2: ", s2, "h2: ", h2)
print("sA3: ", sA3, "QA3: ", QA3, "s3: ", s3, "h3: ", h3)
print("sA4: ", sA4, "QA4: ", QA4, "s4: ", s4, "h4: ", h4)
print("sA5: ", sA5, "QA5: ", QA5, "s5: ", s5, "h5: ", h5)
print("sA6: ", sA6, "QA6: ", QA6, "s6: ", s6, "h6: ", h6)
print("sA7: ", sA7, "QA7: ", QA7, "s7: ", s7, "h7: ", h7)
print("sA8: ", sA8, "QA8: ", QA8, "s8: ", s8, "h8: ", h8)
print("sA9: ", sA9, "QA9: ", QA9, "s9: ", s9, "h9: ", h9)

sA0:  25093761799949614643317388267563277448736513418273664080305291875960916104927 QA0:  (0xe26578142620c2918d0f02073ff145bcfaef2c2aaeba0f1e00fb27b7a095c734 , 0xfc4adc44ecac4d46a9e1f760a780a1c624dc5be321277629e838de749cc624a0) s0:  21502281950839758237085942144349217574482913905047276261005010734414668059614 h0:  107701365619184992967826264015781174951581482835526446422960005719838076081642
sA1:  48303355567975278662487561155102541139962138943007014363497453018153306745517 QA1:  (0xfa372d7392d898520fa2b5718f667de6359ebc637592a69e0077bdf1fde9b4cf , 0xea8ed64e85939735e60581359189e8f4785d6631c5e14e7c0f4f5eef0065a032) s1:  108307206788401921906731110323985095354937480075224312479029002097074342179845 h1:  89817871765069306769348081573745960688367390188556472096213932214054024353930
sA2:  62395255839246821816414398109033603927986411765354137656585067724306534012023 QA2:  (0x3f44353159e66e3f7fe05388d77aad85fbe8757b0b881593927bb2c4a74256b2 , 0x54a028c89527ba566247032a37c152e5033d7147812e840d

In [ ]:
#@title Varsayılan başlık metni

###delete ephemeral keys
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n


mes = {'ID': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "RstEKey"), json = mes)
print(response.json())

200


In [ ]:
#send ephemeral key
mes = {'ID': stuID, 'KEYID': 0 , 'QAI.X': QA0.x, 'QAI.Y': QA0.y, 'Si': s0, 'Hi': h0}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 1 , 'QAI.X': QA1.x, 'QAI.Y': QA1.y, 'Si': s1, 'Hi': h1}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 2 , 'QAI.X': QA2.x, 'QAI.Y': QA2.y, 'Si': s2, 'Hi': h2}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 3 , 'QAI.X': QA3.x, 'QAI.Y': QA3.y, 'Si': s3, 'Hi': h3}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 4 , 'QAI.X': QA4.x, 'QAI.Y': QA4.y, 'Si': s4, 'Hi': h4}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 5 , 'QAI.X': QA5.x, 'QAI.Y': QA5.y, 'Si': s5, 'Hi': h5}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 6 , 'QAI.X': QA6.x, 'QAI.Y': QA6.y, 'Si': s6, 'Hi': h6}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 7 , 'QAI.X': QA7.x, 'QAI.Y': QA7.y, 'Si': s7, 'Hi': h7}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 8 , 'QAI.X': QA8.x, 'QAI.Y': QA8.y, 'Si': s8, 'Hi': h8}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())

mes = {'ID': stuID, 'KEYID': 9 , 'QAI.X': QA9.x, 'QAI.Y': QA9.y, 'Si': s9, 'Hi': h9}
response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
print(response.json())



Key registered
Key registered
Key registered
Key registered
Key registered
Key registered
Key registered
Key registered
Key registered
Key registered


In [ ]:
#Check Status
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h=int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n

mes = {'ID_A':25389, 'H': h, 'S': s}
response = requests.get('{}/{}'.format(API_URL, "Status"), json = mes)
print("Status ", response.json())


Status  You have 10 keys remained. You need to send 0 keys
You have 0 unread messages 


In [ ]:
#SEND 5 MESSAGES BELOW

In [ ]:
#sign the other student's ID
stuID_B = 18007 
m  = str(stuID_B)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n

### Get key of the Student B
mes = {'ID_A': stuID, 'ID_B':stuID_B, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqKey"), json = mes)
res = response.json()
print(res)

{'i': '1', 'j': 0, 'QBJ.x': 103703195163179398601388127235546781297469880063966663038834743768373118851546, 'QBJ.y': 2716098169632154481784797254039771933452578313683467665712462938335234225984}


In [ ]:
### Send message to student B
from Crypto.Hash import HMAC, SHA256
#session keys
def session_keys(si, Qj):
  T = si * Qj
  U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
  U = U.encode()
  KENC_AB = (SHA3_256.new(U))
  KENC_AB_ = KENC_AB.digest()
  KMAC_AB = (SHA3_256.new(KENC_AB_))
  KMAC_AB_ = KMAC_AB.digest()
  return KENC_AB_, KMAC_AB_

stuID =  25389
stuID_B = 18007 

msg = b"The world is full of lonely people afraid to make the first move."
QBJ = Point(res.get('QBJ.x') , res.get('QBJ.y'), E)
i = res.get('i') 
j = res.get('j')

#msg = msg.encode()
K_enc, K_mac = session_keys(sA5, QBJ) #send the sAi according to keyID
cipher = AES.new(K_enc, AES.MODE_CTR)
Y1 = cipher.encrypt(msg)
h = HMAC.new(K_mac, digestmod=SHA256)
h.update(Y1) 
ctext = cipher.nonce[0:8] + Y1 + h.digest()
msg = int.from_bytes(ctext, byteorder='big') 

mes = {'ID_A': stuID, 'ID_B':stuID_B, 'I': i, 'J':j, 'MSG': msg}
response = requests.put('{}/{}'.format(API_URL, "SendMsg"), json = mes)
print(response.json())

Message saved


In [ ]:
#sign the other student's ID
stuID_B = 18007 
m  = str(stuID_B)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n

### Get key of the Student B
mes = {'ID_A': stuID, 'ID_B':stuID_B, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqKey"), json = mes)
res = response.json()
print(res)

{'i': '0', 'j': 349, 'QBJ.x': 63227674659011204738108037186018396018599918869915343505114082448804334663832, 'QBJ.y': 25444612731501886473789307595896007520418826516879458390635109161010977807017}


In [ ]:
### Send message to student B
#MESSAGE2
from Crypto.Hash import HMAC, SHA256
#session keys
def session_keys(si, Qj):
  T = si * Qj
  U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
  U = U.encode()
  KENC_AB = (SHA3_256.new(U))
  KENC_AB_ = KENC_AB.digest()
  KMAC_AB = (SHA3_256.new(KENC_AB_))
  KMAC_AB_ = KMAC_AB.digest()
  return KENC_AB_, KMAC_AB_

stuID =  25389
stuID_B = 18007 

msg = "I don’t like sand. It’s all coarse, and rough, and irritating. And it gets everywhere."
QBJ = Point(res.get('QBJ.x') , res.get('QBJ.y'), E)
i = res.get('i') 
j = res.get('j')

msg = msg.encode()
K_enc, K_mac = session_keys(sA0, QBJ)
cipher = AES.new(K_enc, AES.MODE_CTR)
Y1 = cipher.encrypt(msg)
h = HMAC.new(K_mac, digestmod=SHA256)
h.update(Y1) 
ctext = cipher.nonce[0:8] + Y1 + h.digest()
msg = int.from_bytes(ctext, byteorder='big') 

mes = {'ID_A': stuID, 'ID_B':stuID_B, 'I': i, 'J':j, 'MSG': msg}
response = requests.put('{}/{}'.format(API_URL, "SendMsg"), json = mes)
print(response.json())

Message saved


In [ ]:
#sign the other student's ID
stuID_B = 18007 
m  = str(stuID_B)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n

### Get key of the Student B
mes = {'ID_A': stuID, 'ID_B':stuID_B, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqKey"), json = mes)
res = response.json()
print(res)

{'i': '0', 'j': 351, 'QBJ.x': 15597242726441119441755523083818807930110582120968238658685482639193685769411, 'QBJ.y': 19379716234339032294156757386964313026597209422978268133636055654912076464643}


In [ ]:
### Send message to student B
#MESSAGE3
from Crypto.Hash import HMAC, SHA256
#session keys
def session_keys(si, Qj):
  T = si * Qj
  U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
  U = U.encode()
  KENC_AB = (SHA3_256.new(U))
  KENC_AB_ = KENC_AB.digest()
  KMAC_AB = (SHA3_256.new(KENC_AB_))
  KMAC_AB_ = KMAC_AB.digest()
  return KENC_AB_, KMAC_AB_

stuID =  25389
stuID_B = 18007 

msg = "Hate is baggage. Life’s too short to be pissed off all the time. It’s just not worth it."
QBJ = Point(res.get('QBJ.x') , res.get('QBJ.y'), E)
i = res.get('i') 
j = res.get('j')

msg = msg.encode()
K_enc, K_mac = session_keys(sA0, QBJ)
cipher = AES.new(K_enc, AES.MODE_CTR)
Y1 = cipher.encrypt(msg)
h = HMAC.new(K_mac, digestmod=SHA256)
h.update(Y1) 
ctext = cipher.nonce[0:8] + Y1 + h.digest()
msg = int.from_bytes(ctext, byteorder='big') 

mes = {'ID_A': stuID, 'ID_B':stuID_B, 'I': i, 'J':j, 'MSG': msg}
response = requests.put('{}/{}'.format(API_URL, "SendMsg"), json = mes)
print(response.json())

Message saved


In [ ]:
#sign the other student's ID
stuID_B = 18007 
m  = str(stuID_B)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n

### Get key of the Student B
mes = {'ID_A': stuID, 'ID_B':stuID_B, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqKey"), json = mes)
res = response.json()
print(res)

{'i': '0', 'j': 352, 'QBJ.x': 85967109408044363492781770559070201285944879598892857961275257280568113787272, 'QBJ.y': 11940216781105317458435058962555635720353501856931194968462287618022059058836}


In [ ]:
### Send message to student B
#MESSAGE4
from Crypto.Hash import HMAC, SHA256
#session keys
def session_keys(si, Qj):
  T = si * Qj
  U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
  U = U.encode()
  KENC_AB = (SHA3_256.new(U))
  KENC_AB_ = KENC_AB.digest()
  KMAC_AB = (SHA3_256.new(KENC_AB_))
  KMAC_AB_ = KMAC_AB.digest()
  return KENC_AB_, KMAC_AB_

stuID =  25389
stuID_B = 18007 

msg = "Well, sir, it’s this rug I have, it really tied the room together."
QBJ = Point(res.get('QBJ.x') , res.get('QBJ.y'), E)
i = res.get('i') 
j = res.get('j')

msg = msg.encode()
K_enc, K_mac = session_keys(sA0, QBJ)
cipher = AES.new(K_enc, AES.MODE_CTR)
Y1 = cipher.encrypt(msg)
h = HMAC.new(K_mac, digestmod=SHA256)
h.update(Y1) 
ctext = cipher.nonce[0:8] + Y1 + h.digest()
msg = int.from_bytes(ctext, byteorder='big') 

mes = {'ID_A': stuID, 'ID_B':stuID_B, 'I': i, 'J':j, 'MSG': msg}
response = requests.put('{}/{}'.format(API_URL, "SendMsg"), json = mes)
print(response.json())

Message saved


In [ ]:
#sign the other student's ID
stuID_B = 18007 
m  = str(stuID_B)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h = int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n

### Get key of the Student B
mes = {'ID_A': stuID, 'ID_B':stuID_B, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqKey"), json = mes)
res = response.json()
print(res)

{'i': '0', 'j': 353, 'QBJ.x': 23454253900541725271896926582837300095144439666321145422077642767210268559997, 'QBJ.y': 10331157852702186883875574884021457830210317612306027004681693283828176384401}


In [ ]:
### Send message to student B
#MESSAGE5
from Crypto.Hash import HMAC, SHA256
#session keys
def session_keys(si, Qj):
  T = si * Qj
  U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
  U = U.encode()
  KENC_AB = (SHA3_256.new(U))
  KENC_AB_ = KENC_AB.digest()
  KMAC_AB = (SHA3_256.new(KENC_AB_))
  KMAC_AB_ = KMAC_AB.digest()
  return KENC_AB_, KMAC_AB_

stuID =  25389
stuID_B = 18007 

msg = "Love is like taking a dump, Butters. Sometimes it works itself out. But sometimes, you need to give it a nice hard slimy push."
QBJ = Point(res.get('QBJ.x') , res.get('QBJ.y'), E)
i = res.get('i') 
j = res.get('j')

msg = msg.encode()
K_enc, K_mac = session_keys(sA0, QBJ)
cipher = AES.new(K_enc, AES.MODE_CTR)
Y1 = cipher.encrypt(msg)
h = HMAC.new(K_mac, digestmod=SHA256)
h.update(Y1) 
ctext = cipher.nonce[0:8] + Y1 + h.digest()
msg = int.from_bytes(ctext, byteorder='big') 

mes = {'ID_A': stuID, 'ID_B':stuID_B, 'I': i, 'J':j, 'MSG': msg}
response = requests.put('{}/{}'.format(API_URL, "SendMsg"), json = mes)
print(response.json())

Message saved


In [ ]:
#below is for getting messages and decryption

In [ ]:
m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h=int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n 

## Get your message
mes = {'ID_A': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqMsg_PH3"), json = mes)
print(response.json())


{'IDB': 25334, 'KEYID': 0, 'MSG': 26493286382525014353039658353017008619654269021551312353735468380373875132472151775788008204330999266668524601885153727671826420858817305882414039015554051852030909802214024199155952173751972144243964990040534601586552747510170079590653708144062189513376936740966304808458295, 'QBJ.X': 70226067166027348935917514048584975637091949384078777466905023269741900156888, 'QBJ.Y': 86935586120830898412088888942928497822372712191567369965674437201478914479425}


In [ ]:
def session_keys_and_decrypt(message, si, Qj):
  T = si * Qj
  U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
  U = U.encode()
  KENC_AB = (SHA3_256.new(U))
  KENC_AB_ = KENC_AB.digest()
  KMAC_AB = (SHA3_256.new(KENC_AB_))
  KMAC_AB_ = KMAC_AB.digest()
  ctext = message.to_bytes((message.bit_length()+7)//8, byteorder='big')
  mac = ctext[-32:]
  ctext = ctext[0:-32]
  cipher = AES.new(KENC_AB_, AES.MODE_CTR, nonce=ctext[0:8])
  dtext = cipher.decrypt(ctext[8:])
  h = HMAC.new(KMAC_AB_ , digestmod=SHA256)
  h.update(ctext[8:])
  if mac == h.digest():
    print("The message ", message.to_bytes((message.bit_length()+7)//8, byteorder='big'), "is authentic")
    return dtext
  else:
    return "Not authentic"



In [ ]:
def session_keys_and_decrypt(message, si, Qj):
  T = si * Qj
  U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
  U = U.encode()
  KENC_AB = (SHA3_256.new(U))
  KENC_AB_ = KENC_AB.digest()
  KMAC_AB = (SHA3_256.new(KENC_AB_))
  KMAC_AB_ = KMAC_AB.digest()
  ctext = message.to_bytes((message.bit_length()+7)//8, byteorder='big')
  mac = ctext[-32:]
  ctext = ctext[0:-32]
  cipher = AES.new(KENC_AB_, AES.MODE_CTR, nonce=ctext[0:8])
  dtext = cipher.decrypt(ctext[8:])
  h = HMAC.new(KMAC_AB_ , digestmod=SHA256)
  h.update(ctext[8:])
  if mac == h.digest():
    print("The message ", message.to_bytes((message.bit_length()+7)//8, byteorder='big'), "is authentic")
    return dtext
  else:
    return "Not authentic"

m  = str(stuID)
m = m.encode()
k = randint(1,n-1)
R = k*P
r = (R.x) % n
r = r.to_bytes((r.bit_length()+7)//8, byteorder='big')
T = m+r 
h = (SHA3_256.new(T))
h_= h.digest()
h=int.from_bytes(h_, byteorder='big')
h = h % n
s = ((long_term_private * h) + k)  % n 

## Get your message
mes = {'ID_A': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "ReqMsg_PH3"), json = mes)
print(response.json())

msg = response.get("MSG")
QBJ =  Point(response.get("QBJ.X"), response.get("QBJ.Y"), E)
message = session_keys_and_decrypt(msg,sA0,QBJ) #sA will be determined by the KEYID
h = message
print(h)

NameError: ignored